In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Time
import time

# Random and os for reproducibility
import random
import os

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
import tensorflow as tf

# Processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

# Making keras compatible with scikit learn api
# https://scikit-learn.org/stable/developers/develop.html
from sklearn.base import BaseEstimator, RegressorMixin

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

# Converting Times Series Data to 3D format
from src.utils.split_sequences import split_sequences

# To run cross validation parallelized
from joblib import Parallel, delayed

2024-05-22 08:39:29.143815: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-22 08:39:29.146172: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-22 08:39:29.192420: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-22 08:39:29.193472: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-05-22 08:39:29.982847: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<IPython.core.display.Javascript object>

# Functions and definitions

## Helper functions for blocked time series cross validation

### Convert train/test data to 3D format

In [3]:
def generate_sequences_helper(
    dataset, cement_types, dates=None, timesteps=None, split_by_cement_type=False
):
    index_train = dataset["y_train"].index
    index_test = dataset["y_test"].index

    dataset["y_train"] = dataset["y_train"].reset_index(drop=True)
    dataset["y_test"] = dataset["y_test"].reset_index(drop=True)

    if dates is not None:
        dataset["dates_train"] = dates[index_train].reset_index(drop=True)
        dataset["dates_test"] = dates[index_test].reset_index(drop=True)

    dataset["cement_types_train"] = cement_types.loc[index_train].reset_index(drop=True)
    dataset["cement_types_test"] = cement_types.loc[index_test].reset_index(drop=True)

    dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

    return dataset

<IPython.core.display.Javascript object>

In [4]:
def generate_sequences(dataset, timesteps, split_by_cement_type=False):
    if split_by_cement_type:
        dataset["x_train"], dataset["y_train"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_train"],
                    pd.DataFrame(dataset["x_train"], columns=x.columns),
                    dataset["cement_types_train"],
                    dataset["y_train"],
                ],
                axis=1,
            ),
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_test"],
                    pd.DataFrame(dataset["x_test"], columns=x.columns),
                    dataset["cement_types_test"],
                    dataset["y_test"],
                ],
                axis=1,
            ),
            timesteps,
        )
    else:
        dataset["x_train"], dataset["y_train"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_train"], columns=x.columns),
                    dataset["y_train"],
                ],
                axis=1,
            ).values,
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_test"], columns=x.columns),
                    dataset["y_test"],
                ],
                axis=1,
            ).values,
            timesteps,
        )
    return dataset

<IPython.core.display.Javascript object>

### Data preprocessing

In [5]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [6]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [7]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [8]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(**estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

### Custom Cross Validate

In [9]:
def custom_cross_validate_parallelized(
   Estimator,
   Imputer,
   Transform,
   x,
   y,
   cv,
   timesteps,
   dates=None,
   cement_types=None,
   estimator_params=None,
   imputer_params=None,
   split_by_cement_type=True,
   n_jobs=-1,  # Set the number of parallel jobs, -1 for using all available cores
):
    def process_fold(train_index, test_index, dates, cement_types, x, y):
        dataset = {
           "dates_train": dates[train_index].reset_index(drop=True),
           "cement_types_train": cement_types.loc[train_index].reset_index(drop=True),
           "x_train": x.loc[train_index].reset_index(drop=True),
           "y_train": y[train_index].reset_index(drop=True),
           "dates_test": dates[test_index].reset_index(drop=True),
           "cement_types_test": cement_types.loc[test_index].reset_index(drop=True),
           "x_test": x.loc[test_index].reset_index(drop=True),
           "y_test": y[test_index].reset_index(drop=True),
        }


        set_seeds()

        # Preprocess the dataset
        dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)

        # generate sequences (3D format)
        dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

        # Train and Evaluate the model
        score = train_and_evaluate_model(Estimator, dataset, estimator_params)
        return score

    scores = Parallel(n_jobs=n_jobs)(
    delayed(process_fold)(train_index, test_index, dates, cement_types, x, y)
    for train_index, test_index in cv.split(x)
    )

    # After every iteration metrics results are appended together
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += [value]

    results = [scores_final]
    return results


<IPython.core.display.Javascript object>

## Model Definition

In [10]:
class BidirectionalGRU(BaseEstimator, RegressorMixin):
    def __init__(self):
        self.model = self.get_model()
        self.batch_size = 16
        self.epochs = 300
        self.verbose = 0

    def fit(self, X=None, y=None):
        self.model.fit(
            X, y, batch_size=self.batch_size, epochs=self.epochs, verbose=self.verbose
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.GRU(units=16, activation="relu")
            )
        ),
        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [11]:
def pad_time_series(dataframe, timesteps):
    """
    Pad timeseries with zeros
    """
    df_tmp = pd.DataFrame(
        dict(
            zip(
                dataframe.columns,
                [[0 for _ in range(timesteps - 1)] for _ in range(dataframe.shape[1])],
            )
        )
    )
    df_tmp[DATE] = dataframe[DATE].iloc[0]
    return pd.concat([df_tmp, dataframe], axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

In [12]:
def split_sequences_per_cement_type(dataframe, timesteps, pad=False):
    """
    Create sequences per cement time
    to avoid having parts of the sequence
    of different types of cement.
    """
    if timesteps == 1:
        return split_sequences(
            dataframe.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps
        )

    dates = dataframe[DATE][timesteps - 1 :]
    data = []
    dataframes = []

    for cement_type in CEMENT_TYPES:
        data.append(dataframe[dataframe[cement_type] == 1])
    data.append(dataframe[(dataframe[CEMENT_TYPES] == 0).all(axis=1)])

    for df in data:
        if pad:
            dates = df[DATE].reset_index(drop=True)
            df = pad_time_series(df, timesteps).reset_index(drop=True)
        else:
            dates = df[DATE][timesteps - 1 :].reset_index(drop=True)
        x, y = split_sequences(df.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps)
        x = pd.DataFrame({"Sequences": [sample.tolist() for sample in x]})
        y = pd.DataFrame({"Target": y})
        dataframes.append(pd.concat([dates, x, y], axis=1))

    data = pd.concat(dataframes, axis=0)
    data[DATE] = pd.to_datetime(data[DATE])
    data = data.sort_values(by=DATE).reset_index(drop=True)
    x = data["Sequences"]
    y = data["Target"].values
    x = np.array(x.tolist())

    return x, y

<IPython.core.display.Javascript object>

# Settings for Reproducibility

In [13]:
SEED = 47

<IPython.core.display.Javascript object>

In [14]:
def set_seeds(seed=SEED):
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


# https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed

<IPython.core.display.Javascript object>

In [15]:
def set_global_determinism():
    set_seeds(seed=SEED)

    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

<IPython.core.display.Javascript object>

In [16]:
def suppress_warnings():
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

<IPython.core.display.Javascript object>

In [17]:
def supress_other_stuff():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    # os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

<IPython.core.display.Javascript object>

In [18]:
suppress_warnings()
supress_other_stuff()

<IPython.core.display.Javascript object>

In [19]:
index_to_save = 4

<IPython.core.display.Javascript object>

In [20]:
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}
DATE = "Date"
CEMENT_TYPES = ["Cement_Type_CPIII40", "Cement_Type_CPVARI"]

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [21]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "207",
    "Plant": "AT",
    "Features": "Chemical + CS3",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "BidirectionalGRU",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [22]:
df = pd.read_csv("../../../../../../data/processed/207/at.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [23]:
df_copy = df.copy()
df_copy = pd.get_dummies(data=df_copy, columns=["Cement_Type"], drop_first=True)

df_copy = df_copy.drop(
    [
        "Blaine",
        "#200",
        "#400",
        "Final setting time",
        "Initial setting time",
        "CS7",
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we use all available features

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [24]:
df_copy[CEMENT_TYPES] = df_copy[CEMENT_TYPES].astype(int)
dates = df["Date"].copy()
y = df_copy.pop("CS28")
x = df_copy
df_copy = pd.concat([x, y], axis=1)

<IPython.core.display.Javascript object>

In [25]:
TIMESTEPS_LIST = [1, 7, 14]

<IPython.core.display.Javascript object>

# 1. Long Short Term Memory - BidirectionalGRU

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Timesteps:</b> 1, 3, 5, 7, 10, 15, 20<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [26]:
start = time.time()

repeats = 3
n_splits = 5
TIMESTEPS_LIST = [1, 7, 14]

print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
    x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
    y = df_copy["CS28"]
    scores = custom_cross_validate_parallelized(
        BidirectionalGRU,
        SimpleImputer,
        StandardScaler,
        x,
        y,
        cv,
        timesteps=timesteps,
        dates=dates,
        cement_types=df_copy[CEMENT_TYPES],
        estimator_params=None,
        imputer_params={"strategy": "median"},
        split_by_cement_type=True,
    )
    scores = scores[0]
    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores, METRICS, METRICS_DICT)

    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Repeated KFold"
    results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Repeated Cross Validation:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.757 (0.024)
MAE: 1.357 (0.021)
MAPE: 0.030 (0.000)
R2: 0.731 (0.009)


******
[TEST]
******
RMSE: 1.843 (0.072)
MAE: 1.427 (0.057)
MAPE: 0.032 (0.001)
R2: 0.701 (0.032)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.592 (0.081)
MAE: 1.247 (0.070)
MAPE: 0.028 (0.001)
R2: 0.775 (0.023)


******
[TEST]
******
RMSE: 2.005 (0.110)
MAE: 1.567 (0.086)
MAPE: 0.035 (0.002)
R2: 0.612 (0.056)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.515 (0.082)
MAE: 1.189 (0.073)
MAPE: 0.026 (0.002)
R2: 0.792 (0.023)


******
[TEST]
******
RMSE: 2.082 (0.116)
MAE: 1.643 (0.108)
MAPE: 0.036 (0.002)
R2: 0.536 (0.079)


Minutes Elapsed:  14.53324837287267


<IPython.core.display.Javascript object>

In [27]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features             Model Cross Validation Timesteps RMSE Test  \
                                                                    mean   
0  Chemical + CS3  BidirectionalGRU   Repeated KFold         1  1.843099   
1  Chemical + CS3  BidirectionalGRU   Repeated KFold         7  2.005303   
2  Chemical + CS3  BidirectionalGRU   Repeated KFold        14  2.082366   

             MAE Test           MAPE Test             R2 Test            
        std      mean       std      mean       std      mean       std  
0  0.072070  1.427420  0.056519  0.031703  0.001186  0.701381  0.032247  
1  0.110040  1.567375  0.085693  0.034601  0.001903  0.612365  0.055772  
2  0.115735  1.643348  0.108465  0.036260  0.002468  0.536025  0.079027

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [28]:
start = time.time()

repeats = 3
n_splits = 5
train_size = 0.8
TIMESTEPS_LIST = [1, 7, 14]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]

        cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)

        scores = custom_cross_validate_parallelized(
            BidirectionalGRU,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            cement_types=df_copy[CEMENT_TYPES],
            estimator_params=None,
            imputer_params={"strategy": "median"},
            split_by_cement_type=True,
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "train_size": 0.8}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.778 (0.218)
MAE: 2.167 (0.233)
MAPE: 0.048 (0.005)
R2: -0.193 (0.442)


******
[TEST]
******
RMSE: 3.897 (0.883)
MAE: 3.124 (0.756)
MAPE: 0.068 (0.016)
R2: -1.530 (2.391)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.489 (0.147)
MAE: 1.210 (0.124)
MAPE: 0.027 (0.003)
R2: 0.644 (0.135)


******
[TEST]
******
RMSE: 4.158 (1.502)
MAE: 3.354 (1.029)
MAPE: 0.073 (0.022)
R2: -2.846 (4.852)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.575 (0.241)
MAE: 1.282 (0.221)
MAPE: 0.028 (0.004)
R2: 0.532 (0.273)


******
[TEST]
******
RMSE: 3.325 (1.592)
MAE: 2.813 (1.361)
MAPE: 0.061 (0.029)
R2: -2.237 (3.926)


Minutes Elapsed:  5.535787538687388


<IPython.core.display.Javascript object>

In [29]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features             Model            Cross Validation Timesteps  \
                                                                            
0  Chemical + CS3  BidirectionalGRU  Blocking Time Series Split         1   
1  Chemical + CS3  BidirectionalGRU  Blocking Time Series Split         7   
2  Chemical + CS3  BidirectionalGRU  Blocking Time Series Split        14   
3  Chemical + CS3  BidirectionalGRU              Repeated KFold         1   
4  Chemical + CS3  BidirectionalGRU              Repeated KFold         7   
5  Chemical + CS3  BidirectionalGRU              Repeated KFold        14   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  3.896746  0.883161  3.124118  0.755504  0.068500  0.016240 -1.529919   
1  4.158319  1.501734  3.353886  1.029098  0.073087  0.022124 -2.846011   
2  3.324720  1.591575  2.812686  1.361334  0.060994  0.028780 -2.236678   
3  1.843099  0.072070  1.427420  0.056519  0.031703  0.001186  0.701381   
4  2.005303  0.110040  1.567375  0.085693  0.034601  0.001903  0.612365   
5  2.082366  0.115735  1.643348  0.108465  0.036260  0.002468  0.536025   

             
        std  
0  2.391115  
1  4.852139  
2  3.926309  
3  0.032247  
4  0.055772  
5  0.079027

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [30]:
set_seeds()
start = time.time()
gap = 0
n_splits = 5
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]

        cv = TimeSeriesSplit(
            gap=gap, max_train_size=None, n_splits=n_splits, test_size=None
        )
        scores = custom_cross_validate_parallelized(
            BidirectionalGRU,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            cement_types=df_copy[CEMENT_TYPES],
            estimator_params=None,
            imputer_params={"strategy": "median"},
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "Gap": 0}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.116 (0.300)
MAE: 1.628 (0.195)
MAPE: 0.036 (0.004)
R2: 0.191 (0.410)


******
[TEST]
******
RMSE: 2.798 (1.019)
MAE: 2.243 (0.822)
MAPE: 0.049 (0.016)
R2: -0.387 (1.104)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.817 (0.195)
MAE: 1.459 (0.194)
MAPE: 0.032 (0.004)
R2: 0.387 (0.261)


******
[TEST]
******
RMSE: 3.282 (0.820)
MAE: 2.718 (0.761)
MAPE: 0.060 (0.016)
R2: -0.621 (0.896)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.438 (0.083)
MAE: 1.129 (0.073)
MAPE: 0.025 (0.002)
R2: 0.637 (0.050)


******
[TEST]
******
RMSE: 4.045 (0.557)
MAE: 3.351 (0.554)
MAPE: 0.075 (0.015)
R2: -1.260 (1.189)


Minutes Elapsed:  26.201242236296334


<IPython.core.display.Javascript object>

In [31]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features             Model            Cross Validation Timesteps  \
                                                                            
0  Chemical + CS3  BidirectionalGRU  Blocking Time Series Split         1   
1  Chemical + CS3  BidirectionalGRU  Blocking Time Series Split         7   
2  Chemical + CS3  BidirectionalGRU  Blocking Time Series Split        14   
3  Chemical + CS3  BidirectionalGRU              Repeated KFold         1   
4  Chemical + CS3  BidirectionalGRU              Repeated KFold         7   
5  Chemical + CS3  BidirectionalGRU              Repeated KFold        14   
6  Chemical + CS3  BidirectionalGRU           Time Series Split         1   
7  Chemical + CS3  BidirectionalGRU           Time Series Split         7   
8  Chemical + CS3  BidirectionalGRU           Time Series Split        14   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  3.896746  0.883161  3.124118  0.755504  0.068500  0.016240 -1.529919   
1  4.158319  1.501734  3.353886  1.029098  0.073087  0.022124 -2.846011   
2  3.324720  1.591575  2.812686  1.361334  0.060994  0.028780 -2.236678   
3  1.843099  0.072070  1.427420  0.056519  0.031703  0.001186  0.701381   
4  2.005303  0.110040  1.567375  0.085693  0.034601  0.001903  0.612365   
5  2.082366  0.115735  1.643348  0.108465  0.036260  0.002468  0.536025   
6  2.797807  1.019441  2.242854  0.821584  0.049229  0.015953 -0.386745   
7  3.282198  0.820019  2.717731  0.760712  0.060084  0.016162 -0.621222   
8  4.044728  0.556574  3.351465  0.554432  0.075477  0.014946 -1.260226   

             
        std  
0  2.391115  
1  4.852139  
2  3.926309  
3  0.032247  
4  0.055772  
5  0.079027  
6  1.103798  
7  0.896030  
8  1.189318

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [32]:
start = time.time()
test_size = 0.2
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Out of time Cross Val:")
print(f"Repeats: {repeats}")
print(f"Train: {80}%", f"Test: {20}%")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()

    def parallel_repeats(repeat):
        set_seeds(SEED + repeat)

        scores_final = None

        # Data Splitting
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1).copy()
        y = df_copy["CS28"].copy()

        x_train, x_test, y_train, y_test = train_test_split(
            x, y, test_size=test_size, random_state=SEED, shuffle=False
        )

        # Preprocessing
        imputer = SimpleImputer(strategy="median")
        scaler = StandardScaler()

        x_train = imputer.fit_transform(x_train)
        x_train = scaler.fit_transform(x_train)
        dates_train = dates[: x_train.shape[0]].reset_index(drop=True)
        cement_types_train = df_copy[CEMENT_TYPES][: x_train.shape[0]].reset_index(
            drop=True
        )

        x_test = imputer.transform(x_test)
        x_test = scaler.transform(x_test)
        dates_test = dates[x_train.shape[0] :].reset_index(drop=True)
        cement_types_test = df_copy[CEMENT_TYPES][x_train.shape[0] :].reset_index(
            drop=True
        )

        # Sequence Splitting
        data_train = pd.concat(
            [
                dates_train,
                pd.DataFrame(x_train, columns=x.columns),
                cement_types_train,
                y_train.reset_index(drop=True),
            ],
            axis=1,
        )
        data_test = pd.concat(
            [
                dates_test,
                pd.DataFrame(x_test, columns=x.columns),
                cement_types_test,
                y_test.reset_index(drop=True),
            ],
            axis=1,
        )

        x_train, y_train = split_sequences_per_cement_type(data_train, timesteps)
        x_test, y_test = split_sequences_per_cement_type(data_test, timesteps)

        # Train model and test evalutation
        # Fit model
        pipeline = Pipeline([("estimator", BidirectionalGRU())])
        pipeline.fit(x_train, y_train)

        # Make predictions
        y_train_pred = pipeline.predict(x_train)
        y_test_pred = pipeline.predict(x_test)

        # evaluate predictions
        scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

        return scores_final

    scores = Parallel(n_jobs=5)(
        delayed(parallel_repeats)(repeat) for repeat in range(repeats)
    )
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += value

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    # scores = {key: val[0] for key, val in scores.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Out of time Split"
    results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(
        results_dict_copy, {key: value for key, value in scores_final.items()}
    )
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Out of time Cross Val:
Repeats: 3
Train: 80% Test: 20%



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.826 (0.014)
MAE: 1.424 (0.015)
MAPE: 0.031 (0.000)
R2: 0.562 (0.007)


******
[TEST]
******
RMSE: 1.853 (0.069)
MAE: 1.463 (0.040)
MAPE: 0.035 (0.001)
R2: 0.844 (0.012)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.567 (0.037)
MAE: 1.227 (0.029)
MAPE: 0.027 (0.001)
R2: 0.665 (0.016)


******
[TEST]
******
RMSE: 2.700 (0.154)
MAE: 2.125 (0.137)
MAPE: 0.051 (0.002)
R2: 0.672 (0.038)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.554 (0.057)
MAE: 1.233 (0.043)
MAPE: 0.027 (0.001)
R2: 0.654 (0.025)


******
[TEST]
******
RMSE: 3.780 (0.482)
MAE: 3.055 (0.283)
MAPE: 0.075 (0.008)
R2: 0.363 (0.166)


Minutes Elapsed:  7.673221536477407


<IPython.core.display.Javascript object>

In [33]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)


Features             Model            Cross Validation Timesteps  \
                                                                             
0   Chemical + CS3  BidirectionalGRU  Blocking Time Series Split         1   
1   Chemical + CS3  BidirectionalGRU  Blocking Time Series Split         7   
2   Chemical + CS3  BidirectionalGRU  Blocking Time Series Split        14   
3   Chemical + CS3  BidirectionalGRU           Out of time Split         1   
4   Chemical + CS3  BidirectionalGRU           Out of time Split         7   
5   Chemical + CS3  BidirectionalGRU           Out of time Split        14   
6   Chemical + CS3  BidirectionalGRU              Repeated KFold         1   
7   Chemical + CS3  BidirectionalGRU              Repeated KFold         7   
8   Chemical + CS3  BidirectionalGRU              Repeated KFold        14   
9   Chemical + CS3  BidirectionalGRU           Time Series Split         1   
10  Chemical + CS3  BidirectionalGRU           Time Series Split         7   
11  Chemical + CS3  BidirectionalGRU           Time Series Split        14   

   RMSE Test            MAE Test           MAPE Test             R2 Test  \
        mean       std      mean       std      mean       std      mean   
0   3.896746  0.883161  3.124118  0.755504  0.068500  0.016240 -1.529919   
1   4.158319  1.501734  3.353886  1.029098  0.073087  0.022124 -2.846011   
2   3.324720  1.591575  2.812686  1.361334  0.060994  0.028780 -2.236678   
3   1.852810  0.068801  1.462994  0.040314  0.035095  0.000871  0.843618   
4   2.700378  0.154361  2.124870  0.137182  0.050835  0.002196  0.671563   
5   3.779506  0.482241  3.054799  0.282885  0.074714  0.008309  0.362669   
6   1.843099  0.072070  1.427420  0.056519  0.031703  0.001186  0.701381   
7   2.005303  0.110040  1.567375  0.085693  0.034601  0.001903  0.612365   
8   2.082366  0.115735  1.643348  0.108465  0.036260  0.002468  0.536025   
9   2.797807  1.019441  2.242854  0.821584  0.049229  0.015953 -0.386745   
10  3.282198  0.820019  2.717731  0.760712  0.060084  0.016162 -0.621222   
11  4.044728  0.556574  3.351465  0.554432  0.075477  0.014946 -1.260226   

              
         std  
0   2.391115  
1   4.852139  
2   3.926309  
3   0.011750  
4   0.038183  
5   0.166458  
6   0.032247  
7   0.055772  
8   0.079027  
9   1.103798  
10  0.896030  
11  1.189318

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [34]:
path = "../../../../../../reports/results/local_models/207/at/full/"
filename = f"BidirectionalGRU_results_full_{index_to_save}.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [35]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../../reports/results/local_models/207/at/grouped/"
filename = f"BidirectionalGRU_results_grouped_{index_to_save}.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>